In [9]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

path = r"/Users/yashshah/Downloads/Data/"

buying_costs   = pd.read_csv(path + "Buying_Costs.csv",   index_col=0).values
demand_data    = np.stack([pd.read_csv(path + f"Demand{k}.csv", index_col=0).values
                           for k in range(1, 6)])           # shape (5 stores, 14 days, 3 products)
shipping_costs = pd.read_csv(path + "Shipping Cost.csv", index_col=0).values

T = buying_costs.shape[0]          # (days)
P = buying_costs.shape[1]          # number of products
N = shipping_costs.shape[0]        # number of warehouses
M = shipping_costs.shape[1]        # number of stores


FIXED_ORDER_COST  = 2500
FIXED_SHIP_COST   =   800

IW = [5500, 6500, 7200]               # warehouse space capacities
IS = [  850,   600,   700, 1050, 1350] # store space capacities

inv_cost_wh = [0.10, 0.25, 0.15]         # $/unit/day in warehouse
inv_cost_st = [1.25, 2.25, 1.75]         # $/unit/day in store

size  = [2.0, 1.5, 1.0]                  # space per product unit
init_w = [550, 350, 750]                 # initial WH stock for each product (Milk, Salmon, Strawberry)

BIG_M = 20000


model = gp.Model("Perishable_Goods_Distribution")

# Continuous flows
o   = model.addVars(P, N, T, name="Order")           ##name='Order'
sh  = model.addVars(P, N, M, T, name="Ship")         ##name='Ship'
sw  = model.addVars(P, N, T, name="InvWH")           ##name='StoreWarehouse'
ss  = model.addVars(P, M, T, name="InvStore")        ##name='Store store'

# Binary flags for fixed costs
y_o  = model.addVars(N, T,          vtype=GRB.BINARY, name="IfOrder")
y_sh = model.addVars(N, M, T,       vtype=GRB.BINARY, name="IfShip")

## objective
model.setObjective(
      gp.quicksum(buying_costs[t, p] * o[p, i, t]
                   for p in range(P) for i in range(N) for t in range(T))
    + FIXED_ORDER_COST * gp.quicksum(y_o[i, t] for i in range(N) for t in range(T))
    + gp.quicksum(shipping_costs[i, j] * sh[p, i, j, t]
                   for p in range(P) for i in range(N) for j in range(M) for t in range(T))
    + FIXED_SHIP_COST  * gp.quicksum(y_sh[i, j, t] for i in range(N) for j in range(M) for t in range(T))
    + gp.quicksum(inv_cost_wh[p] * sw[p, i, t]
                   for p in range(P) for i in range(N) for t in range(T))
    + gp.quicksum(inv_cost_st[p] * ss[p, j, t]
                   for p in range(P) for j in range(M) for t in range(T)),
    GRB.MINIMIZE
)


# constraints
# 5‑1  Space capacities
model.addConstrs(gp.quicksum(size[p] * sw[p, i, t] for p in range(P)) <= IW[i]
                 for i in range(N) for t in range(T))
model.addConstrs(gp.quicksum(size[p] * ss[p, j, t] for p in range(P)) <= IS[j]
                 for j in range(M) for t in range(T))

# 5‑2  Warehouse inventory balance  (order arrives next day)
for p in range(P):
    for i in range(N):
        model.addConstr(sw[p, i, 0] ==
                        init_w[p]
                        - gp.quicksum(sh[p, i, j, 0] for j in range(M)))
        model.addConstrs(
            sw[p, i, t] ==
            sw[p, i, t-1]
            + o[p, i, t-1]                          # yesterday’s order arrives today
            - gp.quicksum(sh[p, i, j, t] for j in range(M))
            for t in range(1, T)
        )

# 5‑3  Store inventory balance  (shipments instant)
for p in range(P):
    for j in range(M):
        model.addConstr(ss[p, j, 0] ==
                        gp.quicksum(sh[p, i, j, 0] for i in range(N))
                        - demand_data[j, 0, p])
        model.addConstrs(
            ss[p, j, t] ==
            ss[p, j, t-1]
            + gp.quicksum(sh[p, i, j, t] for i in range(N))
            - demand_data[j, t, p]
            for t in range(1, T)
        )

# 5‑4  Fixed‑cost linking (Big‑M)
model.addConstrs(o[p, i, t] <= BIG_M * y_o[i, t]
                 for p in range(P) for i in range(N) for t in range(T))
model.addConstrs(gp.quicksum(sh[p, i, j, t] for p in range(P)) <= BIG_M * y_sh[i, j, t]
                 for i in range(N) for j in range(M) for t in range(T))


model.optimize()


Set parameter Username
Set parameter LicenseID to value 2625267
Academic license - for non-commercial use only - expires 2026-02-19
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 784 rows, 1344 columns and 3453 nonzeros
Model fingerprint: 0x5956357b
Variable types: 1092 continuous, 252 integer (252 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e-01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 7e+03]
Found heuristic solution: objective 336733.12500
Presolve removed 12 rows and 12 columns
Presolve time: 0.00s
Presolved: 772 rows, 1332 columns, 3429 nonzeros
Variable types: 1083 continuous, 249 integer (249 binary)

Root relaxation: objective 1.235548e+05, 970 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     

In [34]:
# Define product names and colors
products = ['Milk', 'Salmon', 'Strawberry']
colors = {
    'Milk': '\033[36m',        # Teal
    'Strawberry': '\033[31m',  # Mild Red
    'Salmon': '\033[33m'       # Brown/Yellowish
}
reset = '\033[0m'

if model.status == GRB.OPTIMAL:
    order_days = []

    for t in range(T):
        print(f"\nDay {t + 1}:")

        # ───── ORDERS ─────
        print("\nORDERS:")
        placed = False
        for i in range(N):
            for p in range(P):
                q = o[p, i, t].x
                if q > 1e-6:
                    pname = f"{colors[products[p]]}{products[p]}{reset}"
                    print(f"  Order: {pname} to Warehouse {i} = {round(q, 2)}")
                    placed = True
            print()
        if not placed:
            print("  No orders placed on this day.")

        # ───── SHIPMENTS ─────
        print("\nSHIPMENTS:")
        moved = False
        for i in range(N):
            for j in range(M):
                ship_lines = []
                for p in range(P):
                    q = sh[p, i, j, t].x
                    if q > 1e-6:
                        pname = f"{colors[products[p]]}{products[p]}{reset}"
                        ship_lines.append(f"{pname} = {round(q, 2)}")
                if ship_lines:
                    moved = True
                    print(f"\nFrom Warehouse {i} to Store {j}:")
                    for ln in ship_lines:
                        print(f"  {ln}")
        if not moved:
            print("  No shipments on this day.")

        # ───── STORAGE – Warehouses ─────
        print("\nSTORAGE – Warehouses:")
        for i in range(N):
            stock = []
            for p in range(P):
                q = sw[p, i, t].x
                if q > 1e-6:
                    pname = f"{colors[products[p]]}{products[p]}{reset}"
                    stock.append(f"{pname} = {round(q, 2)}")
            if stock:
                print(f"\nWarehouse {i}:")
                for ln in stock:
                    print(f"  {ln}")

        # ───── STORAGE – Stores ─────
        print("\nSTORAGE – Stores:")
        for j in range(M):
            stock = []
            for p in range(P):
                q = ss[p, j, t].x
                if q > 1e-6:
                    pname = f"{colors[products[p]]}{products[p]}{reset}"
                    stock.append(f"{pname} = {round(q, 2)}")
            if stock:
                print(f"\nStore {j}:")
                for ln in stock:
                    print(f"  {ln}")

        if placed:
            order_days.append(t + 1)

    if order_days:
        print(f"\nOrders were placed on the following days: {', '.join(map(str, order_days))}")
    else:
        print("\nNo orders were placed on any day.")
else:
    print("Optimization did not result in an optimal solution.")


Day 1:

ORDERS:
  Order: Milk to Warehouse 0 = 1315.0
  Order: Salmon to Warehouse 0 = 795.0
  Order: Strawberry to Warehouse 0 = 3230.0




SHIPMENTS:

From Warehouse 0 to Store 0:
  Milk = 150.0
  Salmon = 110.0
  Strawberry = 300.0

From Warehouse 0 to Store 2:
  Milk = 180.0
  Salmon = 100.0
  Strawberry = 350.0

From Warehouse 1 to Store 1:
  Milk = 75.0
  Salmon = 65.0
  Strawberry = 150.0

From Warehouse 1 to Store 3:
  Milk = 325.0
  Salmon = 185.0
  Strawberry = 560.0

From Warehouse 2 to Store 4:
  Milk = 300.0
  Salmon = 165.0
  Strawberry = 450.0

STORAGE – Warehouses:

Warehouse 0:
  Milk = 220.0
  Salmon = 140.0
  Strawberry = 100.0

Warehouse 1:
  Milk = 150.0
  Salmon = 100.0
  Strawberry = 40.0

Warehouse 2:
  Milk = 250.0
  Salmon = 185.0
  Strawberry = 300.0

STORAGE – Stores:

Store 2:
  Milk = 80.0
  Salmon = 30.0
  Strawberry = 150.0

Store 3:
  Milk = 145.0
  Salmon = 60.0
  Strawberry = 200.0

Store 4:
  Milk = 55.0

Day 2:

ORDERS:

  Order: Milk to Warehouse 

In [30]:
demand_data[2]

array([[100,  70, 200],
       [ 80,  30, 150],
       [ 80,  40, 120],
       [ 80,  55, 120],
       [ 60,  60, 130],
       [120,  95, 270],
       [140,  90, 320],
       [120,  60, 210],
       [ 60,  45, 120],
       [ 80,  45, 135],
       [ 80,  50, 100],
       [ 90, 110, 160],
       [120, 110, 225],
       [130, 100, 280]])